<h1><center> RECOMMENDATION SYSTEM

## Importing necessary libraries

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow_text as text
import time

In [2]:
import sklearn.metrics as metrics


In [3]:
import re


## Reading the dataset

In [4]:
youtube = pd.read_csv('GB_videos_data.csv')

In [6]:
youtube = pd.read_csv('GB_videos_data.csv')

youtube = youtube.dropna()
youtube = youtube.drop_duplicates(subset = ['description'])
youtube = youtube[['title', 'description']]
youtube.columns = ['Title', 'Description']
youtube['cleaned_description'] = youtube['Description'].apply(lambda x: x.lower())
youtube['cleaned_description'] = youtube['cleaned_description'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [7]:
youtube.shape

(1688, 3)

In [8]:
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1", trainable=True)

## 3) BERT on Description Column

In [9]:


def get_bert_embeddings(text, preprocessor, encoder):
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
    encoder_inputs = preprocessor(text_input)
    outputs = encoder(encoder_inputs)
    embedding_model = tf.keras.Model(text_input, outputs['pooled_output'])
    sentences = tf.constant([text])
    return embedding_model(sentences)


In [10]:
start_time = time.time()
youtube['encodings'] = youtube['cleaned_description'].apply(lambda x: get_bert_embeddings(x, preprocessor, encoder))
print("--- %s seconds ---" % (time.time() - start_time))

--- 96.80083656311035 seconds ---


##### The model took approximately 97 second to run 1370 rows

In [11]:
youtube['encodings'][0]

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[-0.99498975,  0.0361543 , -0.18411422,  0.5946224 , -0.9361536 ,
         0.9088471 , -0.9194194 , -0.9567703 , -0.01298081,  0.1843331 ,
        -0.63755476, -0.05187884,  0.19324712, -0.18263792, -0.01430518,
         0.25525442,  0.89852774,  0.20013627, -0.69245666, -0.8345919 ,
         0.7312774 , -0.10114988, -0.74197346, -0.978764  , -0.999505  ,
         0.04705126, -0.9767469 ,  0.9805024 ,  0.87064797, -0.00470657,
         0.06757554, -0.16520114, -0.30839452, -0.8302583 ,  0.95305634,
         0.8794967 , -0.9858387 ,  0.02577849,  0.22813545, -0.8167166 ,
         0.4067616 ,  0.67985296, -0.2039703 , -0.26841637,  0.3245909 ,
        -0.1106023 ,  0.51197404,  0.94863063, -0.6490156 ,  0.9955804 ,
         0.9421187 , -0.9944358 ,  0.06481232,  0.94674283,  0.9242265 ,
        -0.12704712, -0.03015532, -0.06189512,  0.8896739 , -0.9274301 ,
         0.21583022,  0.824853  ,  0.44138518,  0.16979203,  0.2576085 ,
 

In [17]:
def preprocess_text():
    text = input()
    text = text.lower()
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    return text
  
query_text = preprocess_text()
query_encoding = get_bert_embeddings(query_text, preprocessor, encoder)

youtube['similarity_score'] = youtube['encodings'].apply(lambda x: metrics.pairwise.cosine_similarity(x, query_encoding)[0][0])
df_results = youtube.sort_values(by=['similarity_score'], ascending=False)

Action Hollywood Thrillers


In [18]:
df_results

,Title,Description,cleaned_description,encodings,similarity_score
150,Pyra / Mythra in Smash!!! Nice!!!,neat#SmashBros #Reaction #Episode1,neat smashbros reaction episode1,"((tf.Tensor(-0.99999774, shape=(), dtype=float...",0.960134
1631,i dont know how to play minecraft,my twitch is https://www.twitch.tv/dannygoonza...,my twitch is https www twitch tv dannygoonzale...,"((tf.Tensor(-0.99998415, shape=(), dtype=float...",0.954832
3413,Dana White is Betting $1 Millon that Jake Paul...,Let's make it a $2M bet Dana...Watch me fight ...,let s make it a 2m bet dana watch me fight on ...,"((tf.Tensor(-0.99997956, shape=(), dtype=float...",0.946733
5042,Turning the garden into a trampoline park.,►Making my first trampoline.. and probably the...,making my first trampoline and probably the l...,"((tf.Tensor(-0.999972, shape=(), dtype=float32...",0.942316
1221,I JOINED 1%...,My INSANE introduction montage: https://www.yo...,my insane introduction montage https www youtu...,"((tf.Tensor(-0.99995816, shape=(), dtype=float...",0.941494
...,...,...,...,...,...
19,PARIS SAINT-GERMAIN - AS MONACO (0 - 2) - High...,PARIS SAINT-GERMAIN vs AS MONACO Highlights (0...,paris saint germain vs as monaco highlights 0 ...,"((tf.Tensor(-0.9992719, shape=(), dtype=float3...",0.474990
718,PARIS SAINT-GERMAIN - AS MONACO (0 - 2) - High...,PARIS SAINT-GERMAIN vs AS MONACO Highlights (0...,paris saint germain vs as monaco highlights 0 ...,"((tf.Tensor(-0.9992719, shape=(), dtype=float3...",0.474990
5051,"Game Recap: Lakers 116, Hornets 105","Led by LeBron James 37 points, eight rebounds ...",led by lebron james 37 points eight rebounds a...,"((tf.Tensor(-0.81145954, shape=(), dtype=float...",0.460408
4233,PARIS SAINT-GERMAIN - FC NANTES (1 - 2) - High...,PARIS SAINT-GERMAIN vs FC NANTES Highlights (1...,paris saint germain vs fc nantes highlights 1 ...,"((tf.Tensor(-0.99487877, shape=(), dtype=float...",0.427039


In [19]:
results=df_results[['Title','Description','similarity_score']].copy()

In [20]:
final_results=results.head(10)
final_results

,Title,Description,similarity_score
150,Pyra / Mythra in Smash!!! Nice!!!,neat#SmashBros #Reaction #Episode1,0.960134
1631,i dont know how to play minecraft,my twitch is https://www.twitch.tv/dannygoonza...,0.954832
3413,Dana White is Betting $1 Millon that Jake Paul...,Let's make it a $2M bet Dana...Watch me fight ...,0.946733
5042,Turning the garden into a trampoline park.,►Making my first trampoline.. and probably the...,0.942316
1221,I JOINED 1%...,My INSANE introduction montage: https://www.yo...,0.941494
2240,technoblade fights three dudes,technoblade fights three dudesFOLLOW ME ON TWI...,0.941104
2013,Letting The Person in FRONT of me DECIDE What ...,This is probably the messiest/most awkward vid...,0.934386
5809,Ball Magic #shorts,my first #short 👅https://www.curiositybox.com,0.933478
3421,"Minecraft, But Every Item Is Super...","Minecraft, But Every Item Is Super... (Challen...",0.933328
777,"WILLNE, MEMEULOUS & IMALLEXX (EBOYS SPECIAL) |...",SUBSCRIBE FOR MORE DON'T CRACK WITH WILLNE - h...,0.931442


##### Inference: Even with high similarity scores, the result returned is not accurate. We cannot find titles according to the input tag "Action Hollywood Thrillers"

## 4) SBERT on Description Column

In [21]:
youtube = pd.read_csv('GB_videos_data.csv')
youtube = youtube.drop_duplicates(subset = ['title'])
youtube = youtube[['title', 'description']]
youtube.columns = ['Title', 'Description']

In [22]:
youtube=youtube.reset_index(drop=True)

In [1]:
from sentence_transformers import SentenceTransformer

ModuleNotFoundError: No module named 'sentence_transformers'

In [24]:
sbert=SentenceTransformer('all-MiniLM-L6-v2')

In [25]:
start_time = time.time()
sentence_emb=sbert.encode(youtube['Description'].tolist())
print("--- %s seconds ---" % (time.time() - start_time))


--- 94.92037439346313 seconds ---


In [26]:
inp_val=input('Enter your tag here')
inp=[]
inp.append(inp_val)

Enter your tag hereAction Hollywood Thrillers


In [27]:
inp_emb=sbert.encode(inp)

In [28]:
youtube['SbertEmb']=''
for i in range(0,np.shape(sentence_emb)[0]):
    youtube['SbertEmb'][i]=sentence_emb[i]

In [29]:
youtube['similarity_score']=youtube['SbertEmb'].apply(lambda x:metrics.pairwise.cosine_similarity(np.array(x).reshape(1,-1).tolist(),inp_emb)[0][0])

In [30]:
df_results_SBert=youtube.sort_values(by=['similarity_score'], ascending=False)

In [32]:
df_results_SBert[['Title','Description','similarity_score']].head(10)

,Title,Description,similarity_score
341,Everything Wrong With Snakes On A Plane in 18 ...,This f*cking movie. Next week: Classic comedy ...,0.372724
131,Disney's Cruella | Official Trailer,Brilliant. Bad. A little bit mad. May 2021.Aca...,0.360897
1349,Video Games In Real Life,"From Rust to Among Us, here’s what your favori...",0.347305
135,Demi Lovato: Dancing with the Devil | Official...,Featuring the exclusive first listen to Demi L...,0.340509
1089,Disney’s Cruella | Sneak Peek,"This May, meet the Queen of Mean #Cruella 💋 🖤A...",0.328171
380,Dead by Daylight | Chapter XIX Teaser,Inspiration can often be drawn from the darkes...,0.323026
1070,Post Malone - Hollywood's Bleeding (Live From ...,“Hollywood’s Bleeding” (Live From The 63rd GRA...,0.320625
612,Without Remorse - Official Trailer | Prime Video,"Tom Clancy's Without Remorse, or simply Withou...",0.317350
313,Army of the Dead | Official Teaser | Netflix,A Zack Snyder film. On Netflix May 21. Followi...,0.308234
494,Live in the Moment - Dispo by David Dobrik,Go check out our new version of Dispo: https:/...,0.305523


### Final inferences:
- SBERT is a faster model than BERT
- We need to be careful with the attributes to be taken in the recommendation system
- Similarity score does not represent goodness of fit and cannot be used for validation
- From our implementation, we can say SBERT performs better on noisy data